# <font color=blue>Декораторы (элемент синтаксиса)</font>

Декораторы в Python и примеры их практического использования.

Итак, что же это такое? Для того, чтобы понять, как работают декораторы, в первую очередь следует вспомнить, что функции в python являются объектами, соответственно, их можно возвращать из другой функции или передавать в качестве аргумента. Также следует помнить, что функция в python может быть определена и внутри другой функции.

Вспомнив это, можно смело переходить к декораторам. Декораторы — это, по сути, "обёртки", которые дают нам возможность изменить поведение функции, не изменяя её код.

Создадим свой декоратор "вручную":

In [ ]:
def my_shiny_new_decorator(function_to_decorate):
    # Внутри себя декоратор определяет функцию-"обёртку". Она будет обёрнута вокруг декорируемой,
    # получая возможность исполнять произвольный код до и после неё.
    def the_wrapper_around_the_original_function():
        print("Я - код, который отработает до вызова функции")
        function_to_decorate() # Сама функция
        print("А я - код, срабатывающий после")
    # Вернём эту функцию
    return the_wrapper_around_the_original_function

# Представим теперь, что у нас есть функция, которую мы не планируем больше трогать.
def stand_alone_function():
    print("Я простая одинокая функция, ты ведь не посмеешь меня изменять?")

stand_alone_function()
# Однако, чтобы изменить её поведение, мы можем декорировать её, то есть просто передать декоратору,
# который обернет исходную функцию в любой код, который нам потребуется, и вернёт новую,
# готовую к использованию функцию:
stand_alone_function_decorated = my_shiny_new_decorator(stand_alone_function)
stand_alone_function_decorated()

Возможно мы бы хотели, чтобы каждый раз, во время вызова `stand_alone_function()`, вместо неё вызывалась `stand_alone_function_decorated()`. Для этого просто перезапишем `stand_alone_function()`:

In [ ]:
stand_alone_function = my_shiny_new_decorator(stand_alone_function)
stand_alone_function()

Собственно, это и есть декораторы. Вот так можно было записать предыдущий пример, используя синтаксис декораторов:

In [ ]:
@my_shiny_new_decorator
def another_stand_alone_function():
    print("Оставь меня в покое")

another_stand_alone_function()

То есть, декораторы в python — это просто синтаксическая обертка для конструкций вида:

In [ ]:
another_stand_alone_function = my_shiny_new_decorator(another_stand_alone_function)

Можно использовать несколько декораций для функций:

In [ ]:
def bread(func):
    def wrapper():
        print()
        func()
        print("<\______/>")
    return wrapper

def ingredients(func):
    def wrapper():
        print("#помидоры#")
        func()
        print("~салат~")
    return wrapper

def sandwich(food="--ветчина--"):
    print(food)

sandwich()
sandwich = bread(ingredients(sandwich))
sandwich()

И аналогично через декораторы:

In [ ]:
@bread
@ingredients
def sandwich(food="--ветчина--"):
    print(food)

sandwich()

Не забываем, что так как порядок вызова функций имеет значение, то и порядок проставление декораторов так же имеет значение.

##  Упражнение 1

Напишите функцию, которая получает на вход список чисел и выдает ответ сколько в данном списке четных чисел. Напишите декоратор, который меняет поведение функции следующим образом: если четных чисел нет, то пишет "Нету(" а если их больше 10, то пишет "Очень много"

## <font color=green>Передача декоратором аргументов в функцию</font>

Однако, все декораторы, которые мы рассматривали, не имели одного очень важного функционала — передачи аргументов декорируемой функции. Собственно, это тоже несложно сделать.

Текстовый данные в языке пайтон описываются классом `str`:

In [ ]:
def a_decorator_passing_arguments(function_to_decorate):
    def a_wrapper_accepting_arguments(arg1, arg2):
        print("Смотри, что я получил:", arg1, arg2)
        function_to_decorate(arg1, arg2)
    return a_wrapper_accepting_arguments

# Теперь, когда мы вызываем функцию, которую возвращает декоратор, мы вызываем её "обёртку",
# передаём ей аргументы и уже в свою очередь она передаёт их декорируемой функции
@a_decorator_passing_arguments
def print_full_name(first_name, last_name):
    print("Меня зовут", first_name, last_name)

print_full_name("Vasya", "Pupkin")

# <font color=green>Декорирование методов</font>

Один из важных фактов, которые следует понимать, заключается в том, что функции и методы в Python — это практически одно и то же, за исключением того, что методы всегда ожидают первым параметром ссылку на сам объект (`self`). Это значит, что мы можем создавать декораторы для методов точно так же, как и для функций, просто не забывая про `self`.

При этом строка представляет из себя объект-коллекцию и есть возможность получить доступ к отдельным ее элементам по индексу:

In [ ]:
def method_friendly_decorator(method_to_decorate):
    def wrapper(self, lie):
        lie -= 3
        return method_to_decorate(self, lie)
    return wrapper

class Lucy:
    def __init__(self):
        self.age = 32
    @method_friendly_decorator
    def sayYourAge(self, lie):
        print("Мне {} лет, а ты бы сколько дал?".format(self.age + lie))

l = Lucy()
l.sayYourAge(-3)

### Упражнение 2

Воспользуйтесь написанным классом `Vector2D` и методом `__add__()`. Добавьте к нему декоратор, который при вызове метода печатает сообщение вида: (1, 2) + (3, -1) = (2, 1)

In [ ]:
class Vector2D:
    def __init__(self, *args):
        self.set_xy(*args)
            
    def get_xy(self):
        return [self._x, self._y]
    
    def set_xy(self, *args):
        if len(args) == 2:
            self._x = args[0]
            self._y = args[1]
        elif len(args) == 1:
            self._x = args[0][0]
            self._y = args[0][1]
        else:
            raise ValueError("wrong number of arguments. len(args) == {}".format(len(args)))
            
    def __add__(self, other):
        x, y = other.get_xy()
        return Vector2D(self._x + x, self._y + y)
    
    def __repr__(self):
        return self.__class__.__name__ + '([' + ', '.join(map(str, [self._x, self._y])) + '])'
    
a = Vector2D(1, 2)
b = Vector2D(3, 4)
c = a + b
print(a)
print(b)
c

А теперь попробуем написать декоратор, принимающий аргументы:

In [ ]:
def decorator_maker():
    print("Я создаю декораторы! Я буду вызван только раз: когда ты попросишь меня создать декоратор.")
    def my_decorator(func):
        print("Я - декоратор! Я буду вызван только раз: в момент декорирования функции.")
        def wrapped():
            print ("Я - обёртка вокруг декорируемой функции.\n"
                   "Я буду вызвана каждый раз, когда ты вызываешь декорируемую функцию.\n"
                   "Я возвращаю результат работы декорируемой функции.")
            return func()
        print("Я возвращаю обёрнутую функцию.")
        return wrapped
    print("Я возвращаю декоратор.")
    return my_decorator

# Давайте теперь создадим декоратор. Это всего лишь ещё один вызов функции
new_decorator = decorator_maker()
# Теперь декорируем функцию
def decorated_function():
    print("Я - декорируемая функция.")

decorated_function = new_decorator(decorated_function)
# Теперь наконец вызовем функцию:
decorated_function()

Теперь перепишем данный код с помощью декораторов:

In [ ]:
@decorator_maker()
def decorated_function():
    print("Я - декорируемая функция.")

decorated_function()

Вернёмся к аргументам декораторов, ведь, если мы используем функцию, чтобы создавать декораторы "на лету", мы можем передавать ей любые аргументы, верно?

In [ ]:
def decorator_maker_with_arguments(decorator_arg1, decorator_arg2):
    print("Я создаю декораторы! И я получил следующие аргументы:",
           decorator_arg1, decorator_arg2)
    def my_decorator(func):
        print("Я - декоратор. И ты всё же смог передать мне эти аргументы:",
               decorator_arg1, decorator_arg2)
        # Не перепутайте аргументы декораторов с аргументами функций!
        def wrapped(function_arg1, function_arg2):
            print ("Я - обёртка вокруг декорируемой функции.\n"
                   "И я имею доступ ко всем аргументам\n"
                   "\t- и декоратора: {0} {1}\n"
                   "\t- и функции: {2} {3}\n"
                   "Теперь я могу передать нужные аргументы дальше"
                   .format(decorator_arg1, decorator_arg2,
                           function_arg1, function_arg2))
            return func(function_arg1, function_arg2)
        return wrapped
    return my_decorator

@decorator_maker_with_arguments("Леонард", "Шелдон")
def decorated_function_with_arguments(function_arg1, function_arg2):
    print ("Я - декорируемая функция и я знаю только о своих аргументах: {0}"
           " {1}".format(function_arg1, function_arg2))

decorated_function_with_arguments("Раджеш", "Говард")

Таким образом, мы можем передавать декоратору любые аргументы, как обычной функции.

Некоторые особенности работы с декораторами

1. Декораторы несколько замедляют вызов функции, не забывайте об этом.

2. Вы не можете "раздекорировать" функцию. Безусловно, существуют трюки, позволяющие создать декоратор, который можно отсоединить от функции, но это плохая практика. Правильнее будет запомнить, что если функция декорирована — это не отменить.

3. Декораторы оборачивают функции, что может затруднить отладку.

## <font color=green>Модуль `logging`</font>
Для решения следующего упражнения используйте модуль [`logging`](https://docs.python.org/3/howto/logging.html#logging-basic-tutorial).  Модуль предоставляет богатый инструментарий для контроля за работой приложения:

- вывод в консоль (поток stderr) или файл сообщений;

- поддержка сообщений разных уровней важности: DEBUG, INFO, WARNING, ERROR, CRITICAL, возможность добавления собственных уровней важности;

- фильтрация сообщений по степени важности: только сообщения с достаточным уровнем важности будут выведены;

- логирование имени модуля, из которого пришло сообщение;

- инструменты для записи даты и времени;

- инструменты для логгирования названия модуля, из которого вышло сообщение

In [ ]:
def try_all_log_levels():
    logging.debug("debug message")
    logging.info("my info")
    logging.warning("my warning")
    logging.error("my error")
    logging.critical("my critical error")

In [ ]:
import logging
# По умолчанию уровень логирования - WARNING
try_all_log_levels()

Перезапустите ноутбук и попробуйте

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)
try_all_log_levels()

Можно задать файл, в который будут писаться логи и определить их формат.

Перезапустите ноутбук

In [ ]:
import logging
import os

# Каталог, в котором будет файл с логами должен существовать
os.mkdir('logs')
file_name = 'logs/example.log'

logging.basicConfig(
    filename=file_name,
    level=logging.INFO,
    filemode='w',  #  по умолчанию файйл открывается в режиме 'a'
    format='%(asctime)s - %(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S',
)
logging.info("my info")
logging.warning("my warning")

with open(file_name, 'r') as f:
    print(f.read())

### Упражнение 3

Напишите декоратор, который принимает в качестве аргумента путь к файлу. Если данный декоратор добавить к функции, то в указанный файл будет логироваться информация вида: 

1. Время вызова функции 

2. Входящие аргументы 

3. Ответ return (если есть, если нет то логгировать '-') 

4. Время завершения работы функции 

5. Время работы функции

## <font color=green>Метод  `__call__()`</font>

Некоторые объекты можно "вызывать", передавая им при этом аргументы в круглых скобках. Что объект был вызываемым (callable) нужно, чтобы у него был метод `__call__()`. Например функции являются вызываемыми объектами, а строки - нет.

In [ ]:
def f():
    pass

f()
print(f.__call__, end='\n'*2)

In [ ]:
s = 'abc'
s()

In [ ]:
s.__call__

Для того чтобы сделать экземпляры класса вызываемыми, необходимо добавить в класс метод `__call__()`.

In [ ]:
def apply_op(arg1, arg2, op):
    if op == '+':
        res = arg1 + arg2
    elif op == '-':
        res = arg1 - arg2
    elif op == '*':
        res = arg1 * arg2
    else:
        res = arg1 / arg2
    return res


def compute_postfix(expr):
    stack = []
    expr = expr.split()
    for w in expr:
        if w in '*/+-':
            arg2 = stack.pop()
            arg1 = stack.pop()
            stack.append(apply_op(arg1, arg2, w))
        else:
            stack.append(float(w))
    return stack.pop()


class Calculator:
    def __init__(self, notation):
        self._notation = notation
        
    def __call__(self, expr):
        if self._notation == 'infix':
            return eval(expr)
        elif self._notation == 'postfix':
            return compute_postfix(expr)
        
        
calc1 = Calculator('postfix')
print(calc1('1 2 +'))
calc2 = Calculator('infix')
print(calc2('1 - 2'))

### Упражнение 4

 Решите упражнение 3 с помощью класса.

# <font color=blue> Паттерн проектирования Adapter</font>

Паттерн адаптер — структурный шаблон. Предназначен для обеспечения совместимости объекта системе. По своему принципу подобен переходнику: например у вас есть объект (ноутбук, рассчитанный на подключение к американской розетке) и система (европейская розетка с элестричеством). Тогда — для присоединения объекта к системе необходим адаптер (переходник с европейской розетки на американскую).

<img src="https://upload.wikimedia.org/wikipedia/ru/thumb/0/04/Adapter_pattern.svg/627px-Adapter_pattern.svg.png">

### Пример

In [ ]:
import re
from abc import ABC, abstractmethod

_text = '''
Design Patterns: Elements of Reusable Object-Oriented Software is a software
engineering book describing software design patterns. The book's authors are
Erich Gamma, Richard Helm, Ralph Johnson and John Vlissides with a foreword by
Grady Booch. The book is divided into two parts, with the first two chapters
exploring the capabilities and pitfalls of object-oriented programming, and
the remaining chapters describing 23 classic software design patterns. The
book includes examples in C++ and Smalltalk.
It has been influential to the field of software engineering and is regarded
as an important source for object-oriented design theory and practice. More
than 500,000 copies have been sold in English and in 13 other languages.
The authors are often referred to as the Gang of Four (GoF).
'''

class System:
    ''' Класс, представляющий систему '''
    def __init__(self, text):
        tmp = re.sub(r'\W', ' ', text.lower())
        tmp = re.sub(r' +', ' ', tmp).strip()
        self.text = tmp

    def get_processed_text(self, processor):
        ''' Метод, требующий на вход класс-обработчик '''
        result = processor.process_text(self.text) # Вызов метода обработчика
        print('\n'.join(map(str, result))) # печать результата

class TextProcessor(ABC):
    ''' Абстрактный интерфейс обработчика '''
    @abstractmethod
    def process_text(self, text):
        ''' Здесь должен быть обработчик '''
        pass

class WordCounter:
    ''' Обработчик, несовместимый с основной системой '''
    def count_words(self, text):
        ''' Считает сколько раз встретилось каждое слово текста'''
        self.__words = dict()
        for word in text.split():
            self.__words[word] = self.__words.get(word, 0) + 1

    def get_count(self, word):
        ''' Возвращает количество вхождений '''
        return self.__words.get(word, 0)

    def get_all_words(self):
        ''' Возвращает копию всего словоря слов '''
        return self.__words.copy()

class WordCounterAdapter(TextProcessor):
    ''' Адаптер к обработчику '''
    def __init__(self, adaptee):
        ''' В конструкторе указывается, к какому объекту следует подключить адаптер '''
        self.adaptee = adaptee

    def process_text(self, text):
        ''' Реализация интерфейса обработчика, требуемого системой.'''
        self.adaptee.count_words(text)
        words = self.adaptee.get_all_words().keys()
        return sorted(words,
                      key = lambda x: self.adaptee.get_count(x),
                      reverse = True)

# Создаём систему
system = System(_text)
# Создаём объект - "считатель слов"
counter = WordCounter()
# Подключаем адаптер к обекту
adapter = WordCounterAdapter(counter)
# Запускаем обработчик в системе, через адаптер
system.get_processed_text(adapter)

### Упражнение 5. Адаптер

Вам нужно написать адаптер, который позволил бы использовать найденный вами класс совместно с вашей системой.

Интерфейс класса выглядит следующим образом:

In [ ]:
class Light:
    def __init__(self, dim):
        self.dim = dim
        self.grid = [[0 for _ in range(dim[0])] for _ in range(dim[1])]
        self.lights = []
        self.obstacles = []

    def set_dim(self, dim):
        self.dim = dim
        self.grid = [[0 for _ in range(dim[0])] for _ in range(dim[1])]

    def set_lights(self, lights):
        self.lights = lights
        self.generate_lights()

    def set_obstacles(self, obstacles):
        self.obstacles = obstacles
        self.generate_lights()

    def generate_lights(self):
        return self.grid.copy()

Интерфейс системы выглядит следующим образом:

In [ ]:
class System:
    def __init__(self):
        self.map = self.grid = [[0 for _ in range(30)] for _ in range(20)]
        self.map[5][7] = 1 # Источники света
        self.map[5][2] = -1 # Стены

    def get_lightening(self, light_mapper):
        self.lightmap = light_mapper.lighten(self.map)

Класс `Light` создает в методе `__init__()` поле заданного размера. За размер поля отвечает параметр, представляющий из себя кортеж из 2 чисел. Элемент `dim[1]` отвечает за высоту карты, `dim[0]` за ее ширину. Метод `set_lights()` устанавливает массив источников света с заданными координатами и просчитывает освещение. Метод `set_obstacles()` устанавливает препятствия аналогичным образом. Положение элементов задается списком кортежей. В каждом элементе кортежа хранятся 2 значения: `elem[0]` -- координата по ширине карты и `elem[1]` — координата по высоте соответственно. Метод `generate_lights()` рассчитывает освещенность с учетом источников и препятствий.

Вам необходимо написать адаптер `MappingAdapter`. Прототип класса вам дан в качестве исходного кода.

In [ ]:
class MappingAdapter:
    def __init__(self, adaptee):
        pass

    def lighten(self, grid):
        pass